# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [373]:
problem = '''You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem. '''

## 2. Generate the mathematical model

In [374]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [375]:
#Show the resopnse in a formatted way
Markdown(response.text)

Let's think step by step to formulate a mathematical optimization model for this problem.

**Sets:**

*  $I$: Set of items, indexed by $i$.
*  $B$: Set of boxes, indexed by $j$.

**Parameters:**

*  $w_i$: Weight of item $i$,  $\forall i \in I$.
*  $C$: Capacity of each box.
*  $N$: Maximum number of boxes available.

**Decision Variables:**
* $x_{ij}$: Binary variable equal to 1 if item  $i$ is packed in box $j$, 0 otherwise, $\forall i \in I, \forall j \in B$.

**Objective Function:** 
Minimize the number of boxes used.

(1) $Minimize \sum_{j \in B} \left( 1 - \prod_{i \in I}(1 - x_{ij}) \right)$

The objective function (1) is nonlinear and aims to minimize the number of boxes used. It uses a product to identify boxes that have at least one item assigned to them, thereby counting the boxes used.

**Constraints:**

(2) $\sum_{j \in B} x_{ij} = 1, \forall i \in I$

Constraint (2) ensures that each item is packed into exactly one box.

(3) $\sum_{i \in I} w_i x_{ij} \leq C, \forall j \in B$

Constraint (3) ensures that the total weight of items packed in each box does not exceed the box capacity.

(4) $\sum_{j \in B} \left( 1 - \prod_{i \in I}(1 - x_{ij}) \right) \leq N$

Constraint (4) ensures that the total number of boxes used does not exceed the available number of boxes. 

This model is a nonlinear binary optimization model due to the objective function and constraints being nonlinear in the decision variable. This type of model can be solved by specialized optimization solvers.


## 3. Generate the pyomo code

In [376]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [377]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data
items = ['A', 'B', 'C', 'D', 'E']  # Set of items
boxes = [1, 2, 3, 4]  # Set of boxes - assuming a maximum of 4 boxes available
weights = {'A': 10, 'B': 15, 'C': 8, 'D': 5, 'E': 12} # Weight of each item
capacity = 25  # Capacity of each box

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)  
model.B = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=len(boxes))  # Maximum number of boxes available

# Decision Variables
model.x = pyo.Var(model.I, model.B, within=pyo.Binary)

# Objective Function
def obj_rule(model):
    return sum(1 - pyo.prod(1 - model.x[i, j] for i in model.I) for j in model.B)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
# Each item is packed into exactly one box
def one_box_rule(model, i):
    return sum(model.x[i, j] for j in model.B) == 1
model.one_box = pyo.Constraint(model.I, rule=one_box_rule)

# Box capacity constraint
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C
model.capacity = pyo.Constraint(model.B, rule=capacity_rule)

# Maximum number of boxes constraint
def max_boxes_rule(model):
    return sum(1 - pyo.prod(1 - model.x[i, j] for i in model.I) for j in model.B) <= model.N
model.max_boxes = pyo.Constraint(rule=max_boxes_rule)

# Solve the model
solver = pyo.SolverFactory('gurobi')  # You can choose a different solver if needed
results = solver.solve(model)

# Display results
print(results)

# Print the solution - which item is packed in which box
for j in model.B:
    for i in model.I:
        if pyo.value(model.x[i, j]) == 1:
            print(f"Item {i} is packed in Box {j}")
```

**Sample data is used for:**

* **Sets:** `items` (set of items) and `boxes` (set of boxes).
* **Parameters:** `weights` (weight of each item) and `capacity` (capacity of each box).

This code defines the sets, parameters, variables, objective function, and constraints based on your mathematical formulation. It then uses a solver (in this case, Gurobi) to find the optimal solution for the given data. 

Remember that you'll need to have Pyomo and a suitable solver (like Gurobi, CPLEX, or GLPK) installed in your environment to run this code. 


## 4. Input problem data and try running the generated code

In [435]:
import pyomo.environ as pyo

# Sample Data
items = range(1,25) # Set of items
boxes = range(1,15) # Set of boxes - assuming a maximum of 4 boxes available
weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
capacity = 9  # Capacity of each box

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)  
model.B = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=len(boxes))  # Maximum number of boxes available

# Decision Variables
model.x = pyo.Var(model.I, model.B, within=pyo.Binary)

# Objective Function
def obj_rule(model):
    return sum(1 - pyo.prod(1 - model.x[i, j] for i in model.I) for j in model.B)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
# Each item is packed into exactly one box
def one_box_rule(model, i):
    return sum(model.x[i, j] for j in model.B) == 1
model.one_box = pyo.Constraint(model.I, rule=one_box_rule)

# Box capacity constraint
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C
model.capacity = pyo.Constraint(model.B, rule=capacity_rule)

# Maximum number of boxes constraint
def max_boxes_rule(model):
    return sum(1 - pyo.prod(1 - model.x[i, j] for i in model.I) for j in model.B) <= model.N
model.max_boxes = pyo.Constraint(rule=max_boxes_rule)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # You can choose a different solver if needed
results = solver.solve(model)

# Display results
print(results)

# Print the solution - which item is packed in which box
for j in model.B:
    for i in model.I:
        if pyo.value(model.x[i, j]) == 1:
            print(f"Item {i} is packed in Box {j}")
print(model.obj())


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 39
  Number of variables: 336
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.0493924617767334
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

11.635737821687453


In [439]:
#Making the data for the problem
def BinPackingExample():
    B = 9
    w = [2,3,4,5,6,7,8]
    q = [4,2,6,6,2,2,2]
    s=[]
    for j in range(len(w)):
        for i in range(q[j]):
            s.append(w[j])
    return s,B

#print(BinPackingExample())
def FFD(s, B):
    remain = [B]
    sol = [[]]
    for item in sorted(s, reverse=True):
        for j,free in enumerate(remain):
            if free >= item:
                remain[j] -= item
                sol[j].append(item)
                break
        else:
            sol.append([item])
            remain.append(B-item)
    return sol
s,B = BinPackingExample()
n = len(s)
U = len(FFD(s, B))
print(s) #Weights
print(B) #Max bin capacity
print(n) #Number of boxes available
print(U) #Max number of boxes

[2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 7, 7, 8, 8]
9
24
13


## 5. Correct the code to verify model viability (optional)

In [440]:
model = ConcreteModel()
# Variables
model.x = Var(range(n), range(U), within=Binary, initialize=0)
model.y = Var(range(U), within=Binary, initialize=0)

    # Constraints
    # Each item must be in exactly one bin
def assignment_constraint(model, i):
    return sum(model.x[i, j] for j in range(U)) == 1
model.assignment_constraint = Constraint(range(n), rule=assignment_constraint)

    # The total weight in each bin cannot exceed the bin capacity times the bin use variable
def capacity_constraint(model, j):
    return sum(s[i] * model.x[i, j] for i in range(n)) <= B * model.y[j]
model.capacity_constraint = Constraint(range(U), rule=capacity_constraint)

    # x_ij can only be 1 if y_j is 1 (i.e., if bin j is used)
def linking_constraint(model, i, j):
    return model.x[i, j] <= model.y[j]
model.linking_constraint = Constraint(range(n), range(U), rule=linking_constraint)

    # Objective
model.obj = Objective(expr=sum(model.y[j] for j in range(U)), sense=minimize)

solver = SolverFactory('glpk')
solver.solve(model)

print("Objective Value:", model.obj())

Objective Value: 13.0


## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [441]:
print(response.text)

Let's think step by step to formulate a mathematical optimization model for this problem.

**Sets:**

*  $I$: Set of items, indexed by $i$.
*  $B$: Set of boxes, indexed by $j$.

**Parameters:**

*  $w_i$: Weight of item $i$,  $\forall i \in I$.
*  $C$: Capacity of each box.
*  $N$: Maximum number of boxes available.

**Decision Variables:**
* $x_{ij}$: Binary variable equal to 1 if item  $i$ is packed in box $j$, 0 otherwise, $\forall i \in I, \forall j \in B$.

**Objective Function:** 
Minimize the number of boxes used.

(1) $Minimize \sum_{j \in B} \left( 1 - \prod_{i \in I}(1 - x_{ij}) \right)$

The objective function (1) is nonlinear and aims to minimize the number of boxes used. It uses a product to identify boxes that have at least one item assigned to them, thereby counting the boxes used.

**Constraints:**

(2) $\sum_{j \in B} x_{ij} = 1, \forall i \in I$

Constraint (2) ensures that each item is packed into exactly one box.

(3) $\sum_{i \in I} w_i x_{ij} \leq C, \forall j \i

In [442]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data
items = ['A', 'B', 'C', 'D', 'E']  # Set of items
boxes = [1, 2, 3, 4]  # Set of boxes - assuming a maximum of 4 boxes available
weights = {'A': 10, 'B': 15, 'C': 8, 'D': 5, 'E': 12} # Weight of each item
capacity = 25  # Capacity of each box

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)  
model.B = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=len(boxes))  # Maximum number of boxes available

# Decision Variables
model.x = pyo.Var(model.I, model.B, within=pyo.Binary)

# Objective Function
def obj_rule(model):
    return sum(1 - pyo.prod(1 - model.x[i, j] for i in model.I) for j in model.B)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
# Each item is packed into exactly one box
def one_box_rule(model, i):
    return sum(model.x[i, j] for j in model.B) == 1
